### Step 1: Setup local configuration

Configure workspace and the resource group

In [ ]:
# If needed
#!az login

In [ ]:
#!az configure --defaults workspace=<your workspace> group=<your resource group>

### Step 2: Start the training job

In [ ]:
%%writefile anemoi_training_job.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json

# the 'code' directory must be set up by the user separately 
code: ./replay/atmosphere-subsampled/p0/training
command: >-
  echo ${{inputs.data}} ${{outputs.output_dir}}; ./submit_training_aml.sh ${{inputs.data}} ${{outputs.output_dir}};
environment: azureml:anemoihouse@latest 
resources:
  instance_count: 1
inputs:
  data: 
    type: uri_folder
    mode: ro_mount
    path: azureml:azureml_great_iron_rzt62jgp3b_output_data_data:1
outputs:
  output_dir: 
    type: uri_folder
    mode: upload

distribution:
  type: mpi
  process_count_per_instance: 2
services:
    my_vs_code:
      type: vs_code
    my_jupyter_lab:
      type: jupyter_lab
compute: azureml:Standard-NC80adis-H100-v5
display_name: anemoihouse
experiment_name: anemoihouse
description: anemoihouse


In [ ]:
!az ml job create -f ufs_training_job.yaml

In [ ]:
./submit_training_aml.sh